In [1]:
import os
import requests
import urllib.parse
from dotenv import load_dotenv

# Load environment variables from .env file (where your API key is stored)
load_dotenv()
api = os.getenv('api')  # Riot API key stored in the .env file

# Function to handle API requests and errors
def call_api(url):
    """Handles API requests and error handling."""
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError if the response is an error
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as err:
        print(f"Error occurred: {err}")
    return None

# Function to fetch PUUID
def get_puuid(nome_invocador, tag_invocador):
    """Fetches the PUUID of the summoner based on their summoner name and tag."""
    encoded_name = urllib.parse.quote(nome_invocador)
    api_invocador = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{encoded_name}/{tag_invocador}?api_key={api}"
    
    # Call the API and return the PUUID if successful
    response = call_api(api_invocador)
    if response and 'puuid' in response:
        return response['puuid']
    else:
        print("Failed to retrieve PUUID.")
        return None

# Function to fetch summoner information (Summoner ID, Profile Icon) using PUUID
def get_summoner_info(puuid):
    """Fetches summoner details (Summoner ID, profile icon) using PUUID."""
    api_summoner = f"https://br1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}?api_key={api}"
    
    # Call the API and return summoner details if successful
    response = call_api(api_summoner)
    if response and 'id' in response and 'profileIconId' in response:
        return {
            'summoner_id': response['id'],
            'profile_icon_id': response['profileIconId']
        }
    else:
        print("Failed to retrieve summoner information.")
        return None

# Function to fetch summoner rank using summoner ID
def get_summoner_rank(summoner_id):
    """Fetches the summoner's rank using summoner ID."""
    api_rank = f"https://br1.api.riotgames.com/lol/league/v4/entries/by-summoner/{summoner_id}?api_key={api}"
    
    # Call the API and return rank if successful
    response = call_api(api_rank)
    if response:
        for entry in response:
            if entry['queueType'] == 'RANKED_SOLO_5x5':
                return {
                    'tier': entry['tier'],
                    'rank': entry['rank'],
                    'league_points': entry['leaguePoints'],
                    'wins': entry['wins'],
                    'losses': entry['losses']
                }
    print("Failed to retrieve summoner rank.")
    return None

# Main function to fetch summoner details
if __name__ == "__main__":
    nome_invocador = input("Enter the summoner name: ")
    tag_invocador = input("Enter the summoner tag: ")

    # Step 1: Get PUUID
    puuid = get_puuid(nome_invocador, tag_invocador)
    if puuid:
        print(f"PUUID for {nome_invocador}: {puuid}")

        # Step 2: Get summoner info (Summoner ID and profile icon)
        summoner_info = get_summoner_info(puuid)
        if summoner_info:
            summoner_id = summoner_info['summoner_id']
            profile_icon_id = summoner_info['profile_icon_id']
            print(f"Summoner ID: {summoner_id}, Profile Icon ID: {profile_icon_id}")

            # Step 3: Get summoner rank
            summoner_rank = get_summoner_rank(summoner_id)
            if summoner_rank:
                print(f"Summoner Rank: {summoner_rank['tier']} {summoner_rank['rank']}, LP: {summoner_rank['league_points']}")
                print(f"Wins: {summoner_rank['wins']}, Losses: {summoner_rank['losses']}")
        else:
            print("Could not retrieve summoner information.")
    else:
        print("Could not retrieve PUUID.")


PUUID for JKaah: 4az-WH4th9MyhrwVkobBE72jekfVxOA9cYBW20Oj6f3owqaeEk6oXGH_4OfqQrxZNd2xQmcMdnJK9Q
Summoner ID: oJ4nNqlQMVLLjX-octE77Ig30V0FN5SHzzMOgkbTjZnq-Qs, Profile Icon ID: 6623
Summoner Rank: BRONZE IV, LP: 12
Wins: 49, Losses: 44
